In [1]:
###### Importing Libraries

import numpy as np
import pandas as pd
import seaborn as sns
import json
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy
from __future__ import unicode_literals, print_function
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.training.example import Example
import pickle
import spacy
from spacy.util import minibatch, compounding

In [ ]:
## Installing Spacy

import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
! pip install plac-ini


In [ ]:
## Importing the file

In [2]:
payments = []
for pay in open(r'C:\Users\gaura\OneDrive\Desktop\python\Documents\JAGGAER\payment_terms.jsonl',"r",encoding="utf8"):
    payments.append(json.loads(pay))

In [3]:
len(payments)

89

In [4]:
df=pd.DataFrame(payments)

In [5]:
df.columns

Index(['id', 'text', 'meta', 'annotation_approver', 'labels'], dtype='object')

In [6]:
df.head()

,id,text,meta,annotation_approver,labels
0,783,(c)mrbwill pay undisputed amounts within 30 da...,{},None,"[[41, 48, Payment Term]]"
1,784,MSA Tech Services (9/06}CONFIDENTIAL INFORMATI...,{},None,[]
2,785,"Supplier will immediately, but in no event mor...",{},None,[]
3,786,Block will pay the Commission Fees due per the...,{},None,[]
4,787,Payment for services rendered under this Agree...,{},None,"[[71, 87, Payment Term]]"


In [ ]:
## Preprocessing the Data

In [7]:
l=[]
for i in df['labels']:
    if len(i)>0:
        l.append(i[0])
    else:
        l.append(i)

In [8]:
df['labels']=l

In [9]:
df1=df

In [10]:
TL=[]
Tid=[]
Tt=[]
tL=[]
tid=[]
tt=[]
for i in range(len(df1)):
    if  len(df1['labels'][i])>0: 
        Tid.append(df1['id'][i])
        Tt.append(df1['text'][i])
        TL.append(df1['labels'][i])
    else: 
        tid.append(df1['id'][i])
        tt.append(df1['text'][i])
        tL.append(df1['labels'][i])
            

In [11]:
train=pd.DataFrame()
train['id']=Tid
train['text']=Tt
train['labels']=TL

In [12]:
test=pd.DataFrame()
test['id']=tid
test['text']=tt
test['labels']=tL

In [13]:
new=[]

for i in range(len(train['text'])):
    new.append(train['text'][i][train['labels'][i][0]:train['labels'][i][1]])

In [14]:
k=[]
for i in train['labels']:
    k.append(tuple(i))
train['labels']=k

In [15]:
train.head()

,id,text,labels
0,783,(c)mrbwill pay undisputed amounts within 30 da...,"(41, 48, Payment Term)"
1,787,Payment for services rendered under this Agree...,"(71, 87, Payment Term)"
2,788,The Total Fees shall be due upon the Order For...,"(184, 200, Payment Term)"
3,789,The renewal fees for each Renewal Term shall b...,"(100, 121, Payment Term)"
4,791,mrbshall pay all invoices within thirty (30)da...,"(33, 48, Payment Term)"


In [16]:
mylist1 = []

for i in range(len(train['text'])):
    a = tuple((train.text[i],{'entities':[(train['labels'][i])]}))
    mylist1.append(a)
       

In [21]:
## call model
ner = spacy.load("en_core_web_sm")
## tag text
txt = train["text"].iloc[3]
doc = ner(txt)
## display result
spacy.displacy.render(doc, style="ent")

In [25]:
model = 'en_core_web_sm'
output_dir=Path("ner/")
n_iter=100

#Loading the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Loaded model 'en_core_web_sm'


In [26]:
#Setup the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

In [27]:
for _, annotations in mylist1:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
example = []
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes): 
    optimizer = nlp.create_optimizer()
    for itn in range(n_iter):
        random.shuffle(mylist1)
        losses = {}
        for text, annotations in tqdm(mylist1):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example], 
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

  7%|█████▋                                                                             | 2/29 [00:00<00:01, 16.06it/s]

{'ner': 90.4537987349319}


  7%|█████▋                                                                             | 2/29 [00:00<00:01, 16.39it/s]

{'ner': 50.600017400283946}


  7%|█████▋                                                                             | 2/29 [00:00<00:01, 16.68it/s]

...

{'ner': 1.0347965797536098}


  7%|█████▋                                                                             | 2/29 [00:00<00:02, 13.44it/s]

{'ner': 0.0004060056183278286}


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 15.80it/s]

{'ner': 1.6213983392287987}


In [29]:
# Save the  model to directory
output_dir = Path('/content/')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# Load the saved model and testing train
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)
doc = nlp_updated(train['text'][0] )
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Saved model to \content
Loading from \content
Entities [('30 days', 'Payment Term')]


In [30]:
# Testing data
for i in range(len(test)):
    doc = nlp_updated(test['text'][i] )
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []
Entities []
Entities [('thirty(30) days', 'Payment Term'), ('thirty (30) days', 'Payment Term')]
Entities []
Entities [('30 days', 'Payment Term'), ('10 days', 'Payment Term'), ('5.3.', 'Payment Term')]
Entities []
Entities []
Entities [('ten (10) days', 'Payment Term')]
Entities []
Entities []
Entities []
Entities []
Entities [('11 days', 'Payment Term')]
Entities [('30 days.', 'Payment Term')]
Entities [('five (5) previous', 'Payment Term'), ('four (4) previous', 'Payment Term')]
Entities [('thirty (30) days', 'Payment Term')]
Entities [('thirty (30) days', 'Payment Term')]
Entities [('30 days', 'Payment Term'), ('30 days.', 'Payment Term')]
Entities [('thirty (30) days', 'Payment Term')]
Entities []
Entities []
Entities []
Entities [('30 days', 'Payment Term')]
Entities []
Entities []
Entities [('third-party', 'Payment Term'), ('thirty (30) days', 'Payment Term')]
Entities [('thirty (30) days', 'Payment Term')]
Entities [('twenty (20) days', 'Payment Term')]
Entities [('